In [19]:

import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
from collections import OrderedDict
from itertools import combinations

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.callbacks import TensorBoard
from keras.layers import Input, Dense, LSTM, Dropout
from keras.models import Model

import keras.backend as K
from keras.utils import to_categorical


Using TensorFlow backend.


In [34]:
'''
INPUT
- diagnoses: a dataframe containing patient diagnoses data
- dimmensions: number of dimmensions to return
- unique_patient_identifier: name of the patient identifier in the dataframe
- num_cases: minimum number of cases
- diagnoses_group: name of the diagnoses identifier in the dataframe


OUTPUT:
- embeddings object containing the learnt embeddings for the diagnoses data
'''

def get_embeddings(diagnoses, dimmensions=32, num_cases=500 ,unique_patient_identifier='P_ID', diagnoses_group='DX_GROUP'):
    count = diagnoses.DX_ID.value_counts()
    count = count[count > num_cases]
    
    diagnoses = diagnoses[diagnoses.DX_ID.isin(count[count > num_cases].index)]
    
    y = pd.DataFrame(diagnoses.sort_values('DX_DATE').groupby(['P_ID'])['DX_ID'].apply(list))
    y = y.apply(lambda x: list(OrderedDict.fromkeys(x['DX_ID'])), axis=1)
    
    y = pd.DataFrame(y)
    y.rename(columns={y.columns[0]: "DX_ID" }, inplace = True)
    
    dic = dict.fromkeys(diagnoses['DX_ID'], 0)
    
    j = 0
    for i in dic:
        dic[i] = j
        j += 1
    
    #get all perumtations of unique dx's        
    y['PAIRS'] = y.apply(lambda x: list(combinations([dic[a] for a in x['DX_ID']], 2)) , axis=1)
    y = y.reset_index()
    
    flat_list = [item for sublist in y.PAIRS.tolist() for item in sublist]
    
    print('Pairs Succesfully Created')
    
    x,y = zip(*flat_list)
    
    neural_network(x,y)
    
def neural_network(x,y):
    hot_x = to_categorical(x)
    hot_y = to_categorical(y)
    
    input_dat = Input(shape=(756,))
    encoded = Dense(280, activation='relu')(input_dat)
    encoded = Dense(36)(encoded)
    decoded = Dense(756, activation='softmax')(encoded)
    embeddings = Model(input_dat, decoded)

    encoder = Model(input_dat, encoded)


    embeddings.compile(optimizer='adam', loss='categorical_crossentropy')
    embeddings.summary()
    
    embeddings.fit(hot_x, hot_y,
                epochs=50,
                batch_size=512,
                shuffle=True,
                validation_data=(hot_x, hot_y))
    
    labels = np.identity(80)
    encoded = encoder.predict(labels)

    
    

def getPairs(x):
    return 


class DX_Embeddings:
    def __init__(self):
        pass
    
    # pass in the diagnoses name and this returns an array with the embedding 
    def get_embedding(self, dx_name):
        pass

        


In [35]:
diagnoses = pd.read_csv('dx_100k.tsv', delimiter='\t')


In [32]:
get_embeddings(diagnoses)

Pairs Succesfully Created
{12122523: 0, 12015529: 1, 12125461: 2, 12014097: 3, 12013950: 4, 12019143: 5, 12019113: 6, 12018250: 7, 12153114: 8, 12152895: 9, 12146714: 10, 12134697: 11, 12018877: 12, 12019194: 13, 12153017: 14, 12019572: 15, 12020951: 16, 12127983: 17, 121209117: 18, 12015512: 19, 12138849: 20, 12020609: 21, 12020765: 22, 12154294: 23, 12142874: 24, 12021326: 25, 12124569: 26, 12014928: 27, 12019875: 28, 12019874: 29, 12133896: 30, 12141609: 31, 12019633: 32, 12014948: 33, 12019625: 34, 12015312: 35, 12018614: 36, 12015082: 37, 12121782: 38, 12020776: 39, 12123235: 40, 12015156: 41, 12124934: 42, 12137123: 43, 12018760: 44, 12020738: 45, 1201227: 46, 12014323: 47, 12020816: 48, 12147419: 49, 12124109: 50, 12020621: 51, 12135089: 52, 12014941: 53, 12134159: 54, 12123306: 55, 12123042: 56, 12137202: 57, 12123540: 58, 12014288: 59, 12125077: 60, 12134643: 61, 12147001: 62, 12129108: 63, 12134848: 64, 12014940: 65, 12135675: 66, 121128525: 67, 12124478: 68, 12015508: 69, 12

In [65]:
diagnoses.head()

,P_ID,DX_ID,DX_DATE,DX_NAME,CURRENT_ICD9_LIST,CURRENT_ICD10_LIST,DX_GROUP
1,15,12122523,30413,ATYPICAL CHEST PAIN,786.59,R07.89,"SYMPTOMS, SIGNS, AND ILL-DEFINED CONDITIONS"
3,15,12015529,30413,ESSENTIAL HTN,401.9,I10,CIRCULATORY SYSTEM
5,15,12125461,30439,"SUPRAVENTRICULAR TACHYCARDIA, PAROXYSMAL",427.0,I47.1,CIRCULATORY SYSTEM
6,15,12125461,30439,"SUPRAVENTRICULAR TACHYCARDIA, PAROXYSMAL",427.0,I47.1,CIRCULATORY SYSTEM
7,15,12015529,30439,ESSENTIAL HTN,401.9,I10,CIRCULATORY SYSTEM


In [6]:
from collections import OrderedDict

In [7]:
diagnoses.DX_ID.head()

0    12453646
1    12122523
2    12021313
3    12015529
4    12143339
Name: DX_ID, dtype: int64

In [32]:
count = diagnoses.DX_ID.value_counts()
count[count > 500]

diagnoses = diagnoses[diagnoses.DX_ID.isin(count[count > 500].index)]

In [33]:
y = pd.DataFrame(diagnoses.sort_values('DX_DATE').groupby(['P_ID'])['DX_ID'].apply(list))

In [35]:
y = y.apply(lambda x: list(OrderedDict.fromkeys(x['DX_ID'])), axis=1)

In [37]:
y = pd.DataFrame(y)
y.rename(columns={y.columns[0]: "DX_ID" }, inplace = True)

In [38]:
dic = dict.fromkeys(diagnoses['DX_ID'], 0)

In [39]:
j = 0
for i in dic:
    dic[i] = j
    j += 1

In [40]:
from itertools import combinations

In [41]:
def getPairs(x):
    return list(combinations([dic[a] for a in x['DX_ID']], 2))

In [148]:
sub = y.loc[:100]

In [150]:
sub['pairs'] = sub.apply(getPairs, axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [42]:
y['PAIRS'] = y.apply(getPairs, axis=1)

In [44]:
y.head()

,DX_ID,PAIRS
P_ID,,
15,"[12122523, 12015529, 12014097, 12125461, 12013...","[(0, 1), (0, 3), (0, 2), (0, 4), (0, 5), (0, 6..."
34,"[12141609, 12019633, 12019572, 12014948, 12019...","[(31, 32), (31, 15), (31, 33), (31, 34), (31, ..."
105,"[12135089, 12018614, 12014941, 12134159, 12123...","[(52, 36), (52, 53), (52, 54), (52, 55), (52, ..."
227,"[12124478, 121129783, 12139368, 12134697, 1202...","[(68, 82), (68, 81), (68, 11), (68, 16), (68, ..."
246,"[12015145, 12153114, 12152973, 12018250, 12112...","[(83, 8), (83, 87), (83, 7), (83, 88), (83, 84..."


In [45]:
y = y.reset_index()

In [46]:
y.P_ID.nunique()

84053

In [47]:
flat_list = [item for sublist in y.PAIRS.tolist() for item in sublist]

In [48]:
import keras.backend as K
from keras.utils import to_categorical


In [49]:
x,y = zip(*flat_list)

In [50]:
len(x)

6929731

In [51]:
hot_x = to_categorical(x)
hot_y = to_categorical(y)

In [56]:
# a=dx_data.CURRENT_ICD10_LIST.dropna().apply(lambda x:x.split(','))
# dex = 0
# b = []
# for dx in a:
#     b.append(pd.DataFrame({'P_ID':dx_data.loc[0].P_ID,'DX_DATE':dx_data.loc[0].DX_DATE,'ICD_10':dx}))
#     dex +=1
#     if dex%1000 ==0:
#         print(dex)
np.save('x_data_hot', hot_x)
np.save('y_data_hot', hot_y)


In [54]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.callbacks import TensorBoard
from keras.layers import Input, Dense, LSTM, Dropout
from keras.models import Model


input_dat = Input(shape=(756,))
encoded = Dense(280, activation='relu')(input_dat)
encoded = Dense(36)(encoded)
decoded = Dense(756, activation='softmax')(encoded)
embeddings = Model(input_dat, decoded)

encoder = Model(input_dat, encoded)


embeddings.compile(optimizer='adam', loss='categorical_crossentropy')
embeddings.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 756)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 280)               211960    
_________________________________________________________________
dense_2 (Dense)              (None, 36)                10116     
_________________________________________________________________
dense_3 (Dense)              (None, 756)               27972     
Total params: 250,048
Trainable params: 250,048
Non-trainable params: 0
_________________________________________________________________


In [55]:
embeddings.fit(hot_x, hot_y,
                epochs=50,
                batch_size=512,
                shuffle=True,
                validation_data=(hot_x, hot_y))

Train on 6929731 samples, validate on 6929731 samples
Epoch 1/50
  32256/6929731 [..............................] - ETA: 6:16:10 - loss: 6.3520

KeyboardInterrupt: 

In [455]:
labels = np.identity(80)
encoded = encoder.predict(labels)


In [457]:
keys = dic.keys()

In [459]:
encoded.shape

(80, 36)

In [462]:
keys = list(keys)

In [463]:
np.savetxt("embeddings_data.tsv", encoded, delimiter="\t")
pd.Series(keys).to_csv('labels.tsv', sep='\t', index=False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
